In [1]:
import numpy as np
import pandas as pd

from bokeh.io import show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup, Button, Slider, MultiSelect , TextInput
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, gridplot, row
from bokeh.server.server import Server

def twb_app(doc):
   # output_notebook()
    global current_metadata

    data_dir = '/home/antonis/TWB_translators/'
    metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')
    #metadata = pd.read_pickle('data/english_document_data.pkl')
    
    #metadata['Source_country'] = ['dasdsa' for i in range(len(metadata))]
    #metadata['Format'] = ['dsad3231232s' for i in range(len(metadata))]
    
    # add the extra data and keep only english files
    
    
    extra_features = pd.read_pickle('data/freq_content_updated.pkl')
    english_index = pd.read_pickle('data/english_document_data_topics.pkl').index
    metadata = metadata.loc[english_index] 
    extra_features = extra_features.loc[english_index]
    metadata['Age'] = extra_features['Age']
    metadata['Gender'] = extra_features['Gender']
    metadata['Disabled'] = extra_features['disabl'] > 0.001
    
    doc_type=pd.read_pickle('/home/antonis/TWB_translators/JuSiPy_TWB/docClassif/document_types.pkl')
    metadata=metadata.loc[doc_type.index]
    metadata['Types']=doc_type['type']

    # prepare data source for data table
    source = ColumnDataSource(metadata)
    
    columns = []
    for i in metadata.columns:
        temp = TableColumn(field=i, title=i)
        columns.append(temp)

    data_table = DataTable(source=source, columns=columns, width=1500, height=800)

    # save data state between different widget interactions
    current_metadata = metadata
    
    def gener_helper(x):
        if x == 1:
            return 'Women'
        elif x == 0:
             return 'Men'
        else:
            return 'Unspecified'
        
    def age_helper(x):
        if x == 0:
            return '65+'
        elif x == 1:
             return '0-18'
        elif x == 2:
                return '18-65'
        else:
            return 'Unspecified'
        
    def disabled_helper(x):
        if x == True:
            return 'Disabled'
        else:
            return 'Unspecified'
            
            
    metadata['Gender']=metadata['Gender'].apply(gener_helper)
    metadata['Age']=metadata['Age'].apply(age_helper)
    metadata['Disabled']= metadata['Disabled'].apply(disabled_helper)
    
    
    def text_input_change(attr, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        new_data_view = current_metadata.loc[current_metadata['Filename'].str.contains(new , na=False)]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table
        
    
    
    text_input = TextInput(value="default", title="Tile of a document:")
    text_input.on_change('value', text_input_change)

    
    def dropdown_features_change(attr, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        if int(new) == 0:
            new_data_view = current_metadata.loc[current_metadata['Age'] == '65+']
        elif int(new) == 1:
            new_data_view = current_metadata.loc[current_metadata['Age'] == '0-18']
        elif int(new) ==2:
            new_data_view = current_metadata.loc[current_metadata['Age'] == '18-65']
        elif int(new)== 3:
            new_data_view = current_metadata.loc[current_metadata['Gender'] == 'Men']
        elif int(new) ==4:
            new_data_view = current_metadata.loc[current_metadata['Gender'] == 'Women']
        elif int(new) ==5:
            new_data_view = current_metadata.loc[current_metadata['Disabled'] == 'True']

        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table
 
    menu = [('Age :65+', '0'),('Age :0-18', '1'), ('Age :18-65', '2') , ('Men', '3'), ('Women', '4') ,('Disabled','5') ]
    dropdown_features= Dropdown(label='Document Features', button_type='warning', menu=menu)
    dropdown_features.on_change('value', dropdown_features_change)
    
    # click handler for Age variable
    def dropdown_change(attr, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        new_data_view = current_metadata.loc[current_metadata['Age'] == int(menu[int(new)][1])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    menu = [('65-', '0'),('0-18', '1'), ('18-65', '2') ]
    dropdown = Dropdown(label='Age', button_type='warning', menu=menu)
    dropdown.on_change('value', dropdown_change)

    def dropdown2_change(attr, old, new):
        global current_metadata
        new_data_view = current_metadata.loc[current_metadata['Gender'] == int(menu2[int(new)][1])]
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        col.children[1] = new_data_table
         
    menu2 = [('Men', '0'), ('Women', '1')]
    dropdown2 = Dropdown(label='Gender', button_type='warning', menu=menu2)
    dropdown2.on_change('value', dropdown2_change)
    
    def dropdown_change3(attrname, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = current_metadata.loc[(current_metadata['Source_country'] == menu3[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    temp=metadata.Source_country.unique()
    temp=sorted(temp)
    menu3=[]
    for i in range(len(temp)):
        menu3.append((temp[i], str(i)))
        #menu3.append(temp[i])

    dropdown3 = Dropdown(label='Source country', button_type='warning', menu=menu3)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown3.on_change('value', dropdown_change3)
    
    def dropdown_change4(attrname, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = current_metadata.loc[(current_metadata['Target_country'] == menu3[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table


    dropdown4 = Dropdown(label='Target country', button_type='warning', menu=menu3)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown4.on_change('value', dropdown_change4)
    
    def dropdown_change6(attrname, old, new):
        global current_metadata
        new_data_view = current_metadata.loc[(current_metadata['Types'] == menu6[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table
        
    temp=metadata.Types.unique()
    temp=sorted(temp)
    menu6=[]
    for i in range(len(temp)):
        menu6.append((temp[i], str(i)))    

    dropdown6 = Dropdown(label='Document Type', button_type='warning', menu=menu6)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown6.on_change('value', dropdown_change6)

        
    
    def dropdown_change5(attrname, old, new):
        global current_metadata
        # find appropriates data rows based on new value
        #vcomp = multi_select.value
        new_data_view = current_metadata.loc[(current_metadata['Format'] == menu4[int(new)][0])]
        # update source of data table
        source = ColumnDataSource(new_data_view)
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        current_metadata = new_data_view
        # update the layout of the doc with new data table
        col.children[1] = new_data_table

    temp=metadata.Format.unique()
    temp=sorted(temp)
    menu4=[]
    for i in range(len(temp)):
        menu4.append((temp[i], str(i)))    

    dropdown5 = Dropdown(label='Format', button_type='warning', menu=menu4)
    #multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
    dropdown5.on_change('value', dropdown_change5)

    # This works by returning a list of the checked checkboxes' indexes after a click
    def checkbox_change(new):
        global current_metadata
        # naive way of doing the changes
        '''
        if len(new) == 0:
            print("Gamieste")
            new_data_view = metadata.loc[(metadata['Disabled person'] == False) & (metadata['Unspecified'] == False)]
        elif len(new) == 1:
            new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[['Disabled person', 'Unspecified'].remove(new[0])] == False)]
        else:
            new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[new[1]] == True)]
        '''
        if len(new) == 0:
            new_data_view = current_metadata.loc[current_metadata['Disabled'] == False]
        else:
            new_data_view = current_metadata.loc[current_metadata['Disabled'] == True]
        source = ColumnDataSource(new_data_view)
        current_metadata = new_data_view
        new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
        col.children[1] = new_data_table

    #checkbox_group = CheckboxGroup(labels=['Disabled person','Unspecified'], active=[0, 1])
    checkbox_group = CheckboxGroup(labels=['Disabled person'])
    checkbox_group.on_click(checkbox_change)

    # this is confirmed to work
    def table_reset(new):
        global current_metadata
        current_metadata = metadata
        col.children[1] = data_table

    button = Button(label="Reset", button_type="success")
    button.on_click(table_reset)
    
    def slider_weight_1_change(attrname, old, new):
        slider_weight_3.value=slider_weight_1.value
        '''
        TWB.API.calculate_ranking(news_weight=new)
        '''
        
        #slider_weight_2.value=slider_weight_1.value
        
    slider_weight_1 = Slider(start=0, end=1, value=0.5, step=.1, title="Relevancy_to_news_weight")
    slider_weight_1.on_change('value', slider_weight_1_change)

    def slider_weight_2_change(attrname, old, new):
        slider_weight_1.value=slider_weight_2.value
        #slider_weight_3.value=slider_weight_2.value
        
        
    slider_weight_2 = Slider(start=0, end=1, value=0.5, step=.1, title="Document_features_weight")
    slider_weight_2.on_change('value', slider_weight_2_change)

    def slider_weight_3_change(attrname, old, new):
        #slider_weight_1.value=slider_weight_3.value
        slider_weight_2.value=slider_weight_3.value
        
    slider_weight_3= Slider(start=0, end=1, value=0.5, step=.1, title="Document_content_weight")
    slider_weight_3.on_change('value', slider_weight_3_change)
    
    col = row(column(text_input, dropdown_features, dropdown6,button, slider_weight_1,slider_weight_2,slider_weight_3 ), data_table)
    #col = row(column(text_input, dropdown, dropdown2, dropdown3, dropdown4, dropdown5, dropdown6, checkbox_group,button, slider_weight_1,slider_weight_2,slider_weight_3 ), data_table)

    doc.add_root(col)

In [2]:
#import pandas as pd

#data_dir = '/home/mixalis/Downloads/Translators Without Borders/'
#metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')

#metadata2 = pd.read_pickle('data/english_document_data_topics.pkl')
#metadata.loc[metadata2.index]
#metadata2.info()
#metadata2
#metadata.head(n=3)
#len(metadata)
#metadata.drop('Content',axis=1)
#metadata

#metadata2 = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')
#metadata.columns.difference(metadata2.columns)

In [3]:
# runs the app using bokeh server unsing 1 thread in port 5007
server = Server({'/': twb_app}, num_procs=1, port=8080)
server.start()
server.show('/')

In [ ]:
server.stop()

In [ ]:
'''
import numpy as np
import pandas as pd

from bokeh.io import show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup, Button, Slider, MultiSelect
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, gridplot, row
from bokeh.server.server import Server

output_notebook()   

data_dir = '/home/antonis/TWB_translators/'
metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')

def dropdown_change3(attr,old,new):
    # find appropriates data rows based on new value
    #vcomp = multi_select.value
    print(dropdown3.value)
    new_data_view = metadata.loc[metadata['Source_country'] == menu3[int(dropdown3.value)][0]]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table


source = ColumnDataSource(metadata)
    
temp=metadata.Source_country.unique()
temp=sorted(temp)
menu3=[]
for i in range(len(temp)):
    menu3.append((temp[i], temp[i]+str(i)))
    #menu3.append(temp[i])

dropdown3 = Dropdown(label='Source country', button_type='warning', menu=menu3)
#multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
dropdown3.on_change('value', dropdown_change3)

columns = []
for i in metadata.columns:
    temp = TableColumn(field=i, title=i)
    columns.append(temp)

data_table = DataTable(source=source, columns=columns, width=500, height=200)

col = row(column(dropdown3), data_table)
show(col)
'''

import numpy as np
import pandas as pd

from bokeh.io import show , output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown, CheckboxGroup, Button, Slider, MultiSelect
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, gridplot, row
from bokeh.server.server import Server


   # output_notebook()
global current_metadata

data_dir = '/home/antonis/TWB_translators/'
metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')
    #metadata = pd.read_pickle('data/english_document_data.pkl')
    
    #metadata['Source_country'] = ['dasdsa' for i in range(len(metadata))]
    #metadata['Format'] = ['dsad3231232s' for i in range(len(metadata))]
    
    # add the extra data and keep only english files
#doc_type=pd.read_pickle(/'document_types.pkl')
    
doc_type=pd.read_pickle('/home/antonis/TWB_translators/JuSiPy_TWB/docClassif/document_types.pkl')

english_index = pd.read_pickle('data/english_document_data_topics.pkl').index
metadata = metadata.loc[english_index] 
extra_features = doc_type.loc[english_index]
#metadata['Age'] = extra_features['Age']
#metadata['Gender'] = extra_features['Gender']
#metadata['Disabled'] = extra_features['disabl'] > 0.001




In [ ]:
metadata = pd.read_pickle('data/english_document_data.pkl')
source = ColumnDataSource(metadata)
    
extra_features = pd.read_pickle('data/freq_content_updated.pkl')
metadata['Age'] = extra_features['Age']
metadata['Gender'] = extra_features['Gender']
metadata['Disabled'] = extra_features['disabl '] > 0.001

In [ ]:
columns = []
for i in metadata.columns:
    temp = TableColumn(field=i, title=i)
    columns.append(temp)

data_table = DataTable(source=source, columns=columns, width=1500, height=800)

In [ ]:
# click handler for Age variable
def dropdown_change(attr, old, new):
    # find appropriates data rows based on new value
    new_data_view = metadata.loc[metadata['Age'] == menu[int(new)][0]]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table


menu = [('0-18', '0'), ('18-65', '1'), ('65-', '2')]
dropdown = Dropdown(label='Age', button_type='warning', menu=menu)
dropdown.on_change('value', dropdown_change)

In [ ]:
def dropdown2_change(attr, old, new):
    new_data_view = metadata.loc[metadata['Gender'] == menu2[int(new)][0]]
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    col.children[1] = new_data_table
         
menu2 = [('Men', '0'), ('Women', '1')]
dropdown2 = Dropdown(label='Gender', button_type='warning', menu=menu2)
dropdown2.on_change('value', dropdown2_change)

In [ ]:
def dropdown_change3(attrname, old, new):
    # find appropriates data rows based on new value
    #vcomp = multi_select.value
    new_data_view = metadata.loc[(metadata['Source_country'] == menu3[int(new)][0])]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table

temp=metadata.Source_country.unique()
temp=sorted(temp)
menu3=[]
for i in range(len(temp)):
    menu3.append((temp[i], str(i)))
    #menu3.append(temp[i])

dropdown3 = Dropdown(label='Source country', button_type='warning', menu=menu3)
#multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
dropdown3.on_change('value', dropdown_change3)

In [ ]:
def dropdown_change4(attrname, old, new):
    # find appropriates data rows based on new value
    #vcomp = multi_select.value
    new_data_view = metadata.loc[(metadata['Target_country'] == menu3[int(new)][0])]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table


dropdown4 = Dropdown(label='Target country', button_type='warning', menu=menu3)
#multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
dropdown4.on_change('value', dropdown_change4)

In [ ]:
def dropdown_change5(attrname, old, new):
    # find appropriates data rows based on new value
    #vcomp = multi_select.value
    new_data_view = metadata.loc[(metadata['Format'] == menu4[int(new)][0])]
    # update source of data table
    source = ColumnDataSource(new_data_view)
    new_data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    # update the layout of the doc with new data table
    col.children[1] = new_data_table

temp=metadata.Format.unique()
temp=sorted(temp)
menu4=[]
for i in range(len(temp)):
    menu4.append((temp[i], str(i)))    

dropdown5 = Dropdown(label='Format', button_type='warning', menu=menu4)
#multi_select = MultiSelect(title="Source Country:", value=["GR"],options=menu3)
dropdown5.on_change('value', dropdown_change5)

In [ ]:
# This works by returning a list of the checked checkboxes' indexes after a click
def checkbox_change(new):
    # naive way of doing the changes
    '''
    if len(new) == 0:
        print("Gamieste")
        new_data_view = metadata.loc[(metadata['Disabled person'] == False) & (metadata['Unspecified'] == False)]
    elif len(new) == 1:
        new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[['Disabled person', 'Unspecified'].remove(new[0])] == False)]
    else:
        new_data_view = metadata.loc[(metadata[new[0]] == True) & (metadata[new[1]] == True)]
    '''
    if len(new) == 0:
        new_data_view = metadata.loc[metadata['Disabled person'] == False]
    else:
        new_data_view = metadata.loc[metadata['Disabled person'] == True]
    source = ColumnDataSource(new_data_view)
    data_table = DataTable(source=source, columns=columns, width=1500, height=800)
    
    #col.children[1] = new_data_table

#checkbox_group = CheckboxGroup(labels=['Disabled person','Unspecified'], active=[0, 1])
checkbox_group = CheckboxGroup(labels=['Disabled person'])
checkbox_group.on_click(checkbox_change)

In [ ]:
# this is confirmed to work
def table_reset(new):
    col.children[1] = data_table

button = Button(label="Reset", button_type="success")
button.on_click(table_reset)
    
def slider_weight_1_change(attrname, old, new):
    slider_weight_3.value=slider_weight_1.value
    #slider_weight_2.value=slider_weight_1.value
        
slider_weight_1 = Slider(start=0, end=1, value=0.5, step=.1, title="Relevancy_to_news_weight")
slider_weight_1.on_change('value', slider_weight_1_change)

def slider_weight_2_change(attrname, old, new):
    slider_weight_1.value=slider_weight_2.value
    #slider_weight_3.value=slider_weight_2.value
        
        
slider_weight_2 = Slider(start=0, end=1, value=0.5, step=.1, title="Document_features_weight")
slider_weight_2.on_change('value', slider_weight_2_change)

def slider_weight_3_change(attrname, old, new):
    #slider_weight_1.value=slider_weight_3.value
    slider_weight_2.value=slider_weight_3.value
        
slider_weight_3= Slider(start=0, end=1, value=0.5, step=.1, title="Document_content_weight")
slider_weight_3.on_change('value', slider_weight_3_change)

In [ ]:
col = row(column(dropdown, dropdown2, dropdown3, dropdown4, dropdown5,checkbox_group,button, slider_weight_1,slider_weight_2,slider_weight_3 ), data_table)
doc.add_root(col)

In [ ]:
test = pd.read_pickle('data/freq_content_updated.pkl')

In [ ]:
test.columns